This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [4]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [6]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
nasdaq_url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY
r = requests.get(nasdaq_url)
print(r.status_code)

200


In [8]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response_data = r.json()
# print(response_data)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
# Task 1: collect all data from the year 2017

# We will collect all data points into this data list
data = []
for x in response_data['dataset_data']['data']:
    # check the first four characters in the date to see if the match with '2017'
    date = x[0]
    if date[0:4] == '2017':
        data.append(x)

# 'data' now contains only retrieved data from the year 2017
# print(data)

In [10]:
# Task 2: Convert the returned JSON object into a Python Dictionary

# From this we can see that it is already a Python Dict object
print(type(response_data))

<class 'dict'>


In [11]:
# Column names
names = response_data['dataset_data']['column_names']
for index in range(len(names)):
    print(str(index) + ': ' + names[index])

0: Date
1: Open
2: High
3: Low
4: Close
5: Change
6: Traded Volume
7: Turnover
8: Last Price of the Day
9: Daily Traded Units
10: Daily Turnover


In [12]:
# Task 3: Calculate what the highest and lowest opening prices were for the stock in this period.
# The opening price is stored in elem[1] of each data point in 'data'

# list comprehension collecting all opening prices
opening_prices = [ x[1] for x in data if x[1] ]
print('Highest opening price:', max(opening_prices))
print('Lowest opening price:', min(opening_prices))

Highest opening price: 53.11
Lowest opening price: 34.0


In [17]:
# Task 4: What was the largest change in any one day (based on High and Low price)?
daily_changes = [ abs(x[2] - x[3]) for x in data if x[2] and x[3]]
print('Largest daily change:', round(max(daily_changes), 4))

Largest daily change: 2.81


In [18]:
# Task 5: What was the largest change between any two days (based on Closing Price)?
largest_change = 0
for index in range(len(data)-1):
    largest_change = max(largest_change, abs(data[index][4] - data[index+1][4]))
print('Largest change between two days:', round(largest_change, 4))

Largest change between two days: 2.56


In [19]:
# Task 6: What was the average daily trading volume during this year?

traded_volumes = [ x[6] for x in data if x[6] ]
print('Average daily trading volume:', round(sum(traded_volumes) / len(traded_volumes), 3))

Average daily trading volume: 89124.337


In [20]:
# Task 7: What was the median trading volume during this year?

def get_median(ls):
    """Returns the median value in a list
    """
    ls.sort()
    midpoint = len(ls) // 2
    if len(ls) % 2 == 1:
        # in the case of an odd length, return the middle value
        return ls[midpoint]
    else:
        # in the case of an even length, return the average of the two middle values
        return (ls[midpoint-1] + ls[midpoint]) / 2

print('Median trading volume:', round(get_median(traded_volumes), 3))

Median trading volume: 76286.0
